In [4]:
import sys
sys.path.append('/data/gpuuser1/')
from keras.layers.core import Flatten
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, RepeatVector
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.layers import Merge
from keras.optimizers import SGD
from keras.layers.core import Activation
import numpy as np
import h5py

In [5]:
max_caption_len = 16
vocab_size = 10000

model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
# model.summary()
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1000, activation="softmax"))

# model.load_weights('vgg16_weights.h5')
# print "Model Loaded."

Exception: The shape of the input to "Flatten" is not fully defined (got (0, 7, 512). Make sure to pass a complete "input_shape" or "batch_input_shape" argument to the first layer in your model.

In [5]:
language_model = Sequential()
language_model.add(Embedding(vocab_size, 1024, input_length=max_caption_len))
language_model.add(LSTM(output_dim=1024, return_sequences=True))
language_model.add(TimeDistributed(Dense(1024)))

In [4]:
model.add(RepeatVector(max_caption_len))

full_model = Sequential()
full_model.add(Merge([model, language_model], mode='concat', concat_axis=-1))
full_model.add(LSTM(1024, return_sequences=False))
full_model.add(Dense(vocab_size))
full_model.add(Activation('softmax'))

In [9]:
layers = model.layers
i =0
for layer in layers:
    print layer, i
    i = i+1

<keras.layers.convolutional.ZeroPadding2D object at 0x7ff52e3b8f10> 0
<keras.layers.convolutional.Convolution2D object at 0x7ff52e3b8f90> 1
<keras.layers.convolutional.ZeroPadding2D object at 0x7ff52e049710> 2
<keras.layers.convolutional.Convolution2D object at 0x7ff52e2bc450> 3
<keras.layers.pooling.MaxPooling2D object at 0x7ff52dfd6050> 4
<keras.layers.convolutional.ZeroPadding2D object at 0x7ff52dfa41d0> 5
<keras.layers.convolutional.Convolution2D object at 0x7ff52e008110> 6
<keras.layers.convolutional.ZeroPadding2D object at 0x7ff52df56950> 7
<keras.layers.convolutional.Convolution2D object at 0x7ff52dfb0890> 8
<keras.layers.pooling.MaxPooling2D object at 0x7ff52df54850> 9
<keras.layers.convolutional.ZeroPadding2D object at 0x7ff52df8a850> 10
<keras.layers.convolutional.Convolution2D object at 0x7ff52df8a2d0> 11
<keras.layers.convolutional.ZeroPadding2D object at 0x7ff52df43290> 12
<keras.layers.convolutional.Convolution2D object at 0x7ff52df17190> 13
<keras.layers.convolutional.Ze

In [19]:
f = h5py.File('vgg16_weights.h5')
print f.attrs['nb_layers']
for k in range(f.attrs['nb_layers']):
    if k >16:
        break
#     print k
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    model.layers[k].set_weights(weights)
f.close()

37
